# Notebook: Healthcare No Shows Appointments

Deze notebook werd gemaakt in het kader van opleiding Data Scientist,    
en heeft tot doel om de opgedane kennis te leren gebruiken.   

Deze case hoort thuis bij <B>Supervised Learning</B> en <b>Logistic Regression</B>     

De dataset komt van Kaggle: <a href="https://www.kaggle.com/datasets/iamtanmayshukla/healthcare-no-shows-appointments-dataset"><i>iamtanmayshukla/healthcare-no-shows-appointments-dataset</i></a>

Deze notebook is opgedeeld in verschillende hoofdstukken, telkens aangeduid met een Markdown.

# Some initial settings


Hier volgen een aantal imports om de rest van het notebook vlotter te laten werken.    
Voer deze best uit, zodat de volgende codeblokken niet de volledige import moeten doen en dus sneller laden.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.width', 800)
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
seed=42

# remove future warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore")

# path settings
from pathlib import Path
import os
p = Path()
download_path = p / 'data'
output_path = p / 'output'
images_path = p / 'images'
filename=""

# Create the output directory if it does not exist
if not os.path.exists(output_path):
		os.makedirs(output_path)
# Create the images directory if it does not exist, dit keer op een andere manier dan hierboven
if not images_path.exists():
	images_path.mkdir(parents=True, exist_ok=True)

# Functie om de output van de gemaakte plots te bewaren
def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    path = images_path / f"{fig_name}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


from tqdm.notebook import tqdm
from datetime import datetime
#import datetime
start_time = datetime.now() # Om op het einde de uitvoertijd te berekenen


def print_exec_time(exec_info):
    tqdm.write(f"✅ Uitgevoerd op: {datetime.now():%Y-%m-%d %H:%M:%S}")

get_ipython().events.register("post_run_cell", print_exec_time)

# Get the data from Kaggle

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import zipfile

api = KaggleApi()
api.authenticate()

print(datetime.now())

if 'data\\' in filename: 
    print(f"Bestaat al: {filename}")
    print(f"Het bestand is al gedownload en moet niet meer worden ingelezen.\n")
else:
    # download dataset
    kaggle_dataset_name = 'iamtanmayshukla/healthcare-no-shows-appointments-dataset' # te halen van de Kaggle.com url
    api.dataset_download_files(kaggle_dataset_name, path=download_path, unzip=True)

    # Het gedownloade bestand oproepen vanuit de download map
    files = os.listdir(download_path)

    # neem de eerste file uit de folder
    filename1 = os.path.join(download_path, files[0])
    # Neem de recentste file op basis van de file met de nieuwste modification time
    file_paths = [os.path.join(download_path, file) for file in files]
    filename2 = max(file_paths, key=os.path.getmtime)

    # check filename (en path)
    print(f"\nHet bestand waarmme je verder werkt is: \n 1. {filename1} of \n 2. {filename2}")

    # Kies met welke filename je verder wilt werken
    filename = filename1

    # Lees het bestand in
    df_raw = pd.read_csv(filename)
    display(df_raw.head())


### Uit te voeren om met *verse* data te starten

In [ ]:
# maak een nieuwe kopie van de originele dataset, zonder deze opnieuw te moeten downloaden
dataset = df_raw.copy()
# Snelle controle of de nieuwe dataset goed is ingelezen
print(dataset.shape)
print(dataset.head())

## Profiling

Om een eerste indruk te krijgen over de dataset, maken we gebruik van een profiler.   
In het geëxporteerde bestand /output/profiler.html krijg je hierna een mooi overzicht van alle data waaruit deze dataset is opgebouwd.

### ydata profiling

In [ ]:
##########################
# PROFILER HTML DOCUMENT #
##########################

from ydata_profiling import ProfileReport
import os
import webbrowser

# controleer of er al een rapport is
profiler_file = 'HNSA_1_profiler.html'
profiler_path = os.path.abspath(os.path.join(output_path, profiler_file))
if os.path.exists(profiler_path):
	print(f"profiler betsaat al. Openen...")
else:
	# Generate the profiling report, kies een goede titel
	profile = ProfileReport(dataset, title="Dataset Healthcare No Shows Appointments Report", explorative=True) # explorative=True om ook de correlaties te zien



	# Save the report as an HTML file
	profile.to_file(os.path.join(output_path,profiler_file))

	# Pad naar je bestand
	profiler_path = os.path.abspath(os.path.join(output_path, profiler_file))
	print(f"profiler gemaakt: {profiler_path}")

# Open het bestand in de standaardbrowser
webbrowser.open(f"file://{profiler_path}")



## Exploring

### Elementaire checks

In [ ]:
# aantal rijen printen
print(f"Aantal rijen: {dataset.shape[0]}")
print(f"Aantal kolommen: {dataset.shape[1]}")
print("-----------------")
# een overzicht van de kolommen
print(f"Kolommen: {dataset.columns}")
print("-----------------")
# Check for missing values
if dataset.isnull().values.any():
    print(dataset.isnull().sum()) # aantal missing values per kolom
else:
    print(f"Missing values: GEEN !! ")
print("-----------------")
# Get basic statistics
print(f"Statistics: (voor numerische kolommen) \n")
print(dataset.describe())
print("-----------------")
# Check the data types
print(f"Data types :")
print(dataset.dtypes)
print("-----------------")

# Check unique values in categorical columns
for col in dataset.select_dtypes(include=['object']).columns:
    print(f"{col} unique values: {dataset[col].nunique()}")

print("-----------------")

Vooraleer verder te gaan , kan je best het Profiler rapport bekijken.    
Hierin vind je oa:   
 - overzicht van de dataset
 - Alerts, met vermelding van correlations, imbalances, ...
 - Detail van alle features, , incl statistics, histogram, unieke waarden, missing, ....
 - Interactions
 - Correlations
 - Missing values
 - Sample


Opmerking:   
Er zijn enkele waarden ongebalanceerd en sommige zijn toch wel hoog gecorreleerd.

### Eerst eens de numerische kolommen bekijken

### Voor cleaning

In [ ]:
def plot_num_cols(df):
    # Selecteer numerieke kolommen
    numeric_cols = dataset.select_dtypes(include=['int64', 'float64']).columns

    # Bereken het aantal subplots dat we nodig hebben
    n_plots = len(numeric_cols)

    # Bepaal aantal rijen en kolommen voor de subplots
    ncols = 3  # Aantal kolommen in de matrix
    nrows = (n_plots // ncols) + (n_plots % ncols > 0)  # Het aantal rijen is afhankelijk van het aantal plots

    # Maak de subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))

    # Flatten de assen om makkelijk te itereren
    axes = axes.flatten()

    # Genereer een histogram voor elke numerieke kolom
    for i, col in enumerate(numeric_cols):
        sns.histplot(dataset[col], ax=axes[i], bins=30, kde=True)
        axes[i].set_title(f"Verdeling van {col}")
        axes[i].set_xlabel(col)
        axes[i].set_ylabel("Frequentie")

    # Verberg lege subplots (indien nodig)
    for i in range(n_plots, len(axes)):
        axes[i].axis('off')

    # Vertoon de plot
    plt.tight_layout()
    save_fig("HNSA_1_Histograms_numeric_columns")
    plt.show()


In [ ]:
plot_num_cols(dataset)

# Cleaning

Er zijn geen missing values, duplicates, ...    
Hierdoor moeten we geen rijen schrappen of waarden opvullen.

In [ ]:
# omzetten van PatinetId naar string en verwijder .0 op het einde van de string, dit is geen numerische waarde met betekenis
dataset['PatientId'] = dataset['PatientId'].astype(str).str.replace('.0', '', regex=False)

# omzetten van AppointmentID naar string en verwijder .0 op het einde van de string, dit is geen numerische waarde met betekenis
dataset['AppointmentID'] = dataset['AppointmentID'].astype(str).str.replace('.0', '', regex=False)

# omzetten van ScheduledDay en AppointmentDay naar datetime
dataset['ScheduledDay'] = pd.to_datetime(dataset['ScheduledDay'])
dataset['AppointmentDay'] = pd.to_datetime(dataset['AppointmentDay'])

# Aanmaken van extra kolommen voor de dag van de week, dit zou een invloed kunnen hebben.
dataset['AppointmentDayOfWeek'] = dataset['AppointmentDay'].dt.day_name()
dataset['ScheduledDayOfWeek'] = dataset['ScheduledDay'].dt.day_name()


### NA cleaning

In [ ]:
plot_num_cols(dataset)


In [ ]:
# check uitvoering vorig codeblok
print(dataset.dtypes)
dataset


Opmerking:    
Er blijven nog kolommen over als 'opject'.    
Deze worden later nog omgezet.

# Exploratory Data Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#import matplotlib
#matplotlib.use('TkAgg')  # or 'Qt5Agg'
#%matplotlib inline

# Day of Week verdeling
#twee subplots
plt.figure(figsize=(12, 6))
# Sort days on x-axis
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
plt.subplot(1, 2, 1)
sns.countplot(data=dataset, x='AppointmentDayOfWeek', hue='Showed_up', order=day_order)
plt.title('Appointment Showed up by Day of the Week')
plt.subplot(1, 2, 2)
sns.countplot(data=dataset, x='ScheduledDayOfWeek', hue='Showed_up', order=day_order)
plt.title('Scheduled Day Showed up by Day of the Week')
# Afdrukken van de twee plots naast elkaar
save_fig("HNSA_1_DayOfWeekShowedUp")
plt.show()

In [ ]:
# Age distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='Age', bins=30, hue='Showed_up', multiple='stack')
plt.title('Age Distribution by Show-up Status')
save_fig("HNSA_1_Age_Distribution_by_Show_up_Status")
plt.show()

### Correlatie matrix   
Eerst op de originele data   
Daarna op de versie met de Bool-kolommen omgezet naar 0/1

In [ ]:
# Originele Matrix
import matplotlib
#matplotlib.use('TkAgg')  # or 'Qt5Agg'
#%matplotlib inline
# Selectie kolommen met numerieke waarden
numeric_df_raw = df_raw.select_dtypes(include=[np.number])

# Bereken de correlatie matrix
corr = numeric_df_raw.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
save_fig("HNSA_1_Correlation_Heatmap")
plt.show()

In [ ]:
# NIET MEER VAN TOEPASSING, maar kan misschien nog van pas komen

"""
# Nieuwe Matrix met alle (numerische) kolommen
# wetende dat de Boolean omgezet zijn naar int 1 of 0

# Selectie kolommen met numerieke waarden
numeric_data = dataset.select_dtypes(include=[np.number])

# Bereken de correlatie matrix
corr = numeric_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
save_fig("HNSA_1_Correlation_Heatmap_All")
plt.show()
"""

# START Modelling  

### Split Train & Test

We starten met de bruikbare kolommen.    
Later gaan we de data bewerken en opnieuw enkele modellen toepassen.

In [ ]:
from sklearn.model_selection import train_test_split

# Prepare data for modeling
features = ['Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Date.diff']
target = 'Showed_up'
X = dataset[features]
y = dataset[target]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)


### Predictive Modeling   Logistic Regression    
Baseline model (eenvoudig en interpreteerbaar) 

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
con_matrix = confusion_matrix(y_test, y_pred)


print(f"Accuracy: {accuracy:.4f}\n")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(con_matrix)



In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print(f"Model gebruikt: {model} \n")
print(f"Cross-Validation Scores: {scores}")
print(f"Mean CV Score: {scores.mean():.4f}")

### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the model
model = RandomForestClassifier(random_state=seed)
print(f"Model gebruikt: {model} \n")
# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# More detailed evaluation
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# For ROC-AUC score
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}\n")

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
print(f"False Positive Rate: {fpr}")
print(f"True Positive Rate: {tpr}")
print(f"Thresholds: {thresholds}")
print(len(fpr), len(tpr), len(thresholds))


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print(f"Model gebruikt: {model} \n")
print(f"Cross-Validation Scores: {scores}")
print(f"Mean CV Score: {scores.mean():.4f}")

We zien niet direct een verbetering bij dit model (RandomForestClassifier).     
We gaan nu de meest gebruikte modellen vergelijken, zijnde:    
- Logistic Regression
- DecisionTreeClassifier
- RandomForrestClassifier
- XGBClassifier
- SVC
- KNeighborsClassifier
- MLPClassifier
- AdaBoostClassifier (boosting-techniek, werkt goed bij kleinere datasets)
- GradientBoostingClassifier (populair voor gestructureerde data, zoals in Kaggle-wedstrijden)
- ExtraTreesClassifier (variant van RandomForest met extra randomisatie)
- LGBMClassifier (Lichtgewicht en snel, ideaal voor grote datasets)
- 

In [ ]:
# Hier gaat het gebeuren, meerdere modellen vergelijken

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Functie om modellen te trainen en evalueren
def evaluate_models(X, y):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    # Definieer modellen
    models = {
        "Logistic Regression": LogisticRegression(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "Extra Trees": ExtraTreesClassifier(),
        "AdaBoost": AdaBoostClassifier(),
        "Gradient Boosting": GradientBoostingClassifier(),
        "XGBoost": XGBClassifier(eval_metric='logloss'),
        "LGBM": LGBMClassifier(),
#        "SVM": SVC(probability=True),  # Dit duurde veel te lang om uit te voeren, deze is niet geschikt voor grotere datsets, daarom de volgende LinearSVC
#        "Linear SVM": LinearSVC(),
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "Neural Network": MLPClassifier(max_iter=500),
    }

    # Resultaten opslaan
    scores = {}

    # Train en evalueer elk model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        scores[name] = acc
        print(f"{name}: {acc:.4f}")  # Print de score

    # Plot de scores
    plt.figure(figsize=(12, 6))
    sns.barplot(x=list(scores.keys()), y=list(scores.values()), palette="viridis")
    plt.xlabel("Model")
    plt.ylabel("Accuracy")
    plt.title("Vergelijking van Model Prestaties")
    plt.xticks(rotation=30)
    plt.ylim(0, 1)
    save_fig("Model_Prestaties")
    plt.show()

# Roep de functie aan met de dataset X en y
evaluate_models(X, y)


# Het grotere werk   
We gaan de kolommen 'dagen' en 'neighbourhood' encoden    
Kolom 'Age' scalen    
Dit keer (bijna) ALLE kolommen (behalve Target) hernemen in X

In [ ]:
""" 
# Deze optie laat ik vallen en kies voor OHE, welke hieronder wordt uitgevoerd. 

# Dagen van de week omzetten met OHE via get dummies
days_SDW = {'ScheduledDayOfWeek': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']}
dataset = pd.get_dummies(dataset, columns=days_SDW.keys())
days_ADW = {'AppointmentDayOfWeek': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']}
dataset = pd.get_dummies(dataset, columns=days_ADW.keys())
""" 

from sklearn.preprocessing import OneHotEncoder

# Werken met OHE zelf ipv getdummies:
days = {'ScheduledDayOfWeek': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'],
        'AppointmentDayOfWeek': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']}

enc1 = OneHotEncoder(sparse=False)
encoded_SDW = enc1.fit_transform(dataset[['ScheduledDayOfWeek']])
#encoded_df1 = pd.DataFrame(encoded_SDW, columns=[f'SDW_{col}' for col in enc1.categories_[0]])
encoded_df1 = pd.DataFrame(encoded_SDW, columns=enc1.get_feature_names_out(['ScheduledDayOfWeek']))
dataset = dataset.drop(columns=['ScheduledDayOfWeek'])
enc2 = OneHotEncoder(sparse=False)
encoded_ADW = enc1.fit_transform(dataset[['AppointmentDayOfWeek']])
encoded_df2 = pd.DataFrame(encoded_ADW, columns=enc1.get_feature_names_out(['AppointmentDayOfWeek']))
dataset = dataset.drop(columns=['AppointmentDayOfWeek'])
#Voeg samen met originele dataset
dataset = pd.concat([dataset, encoded_df1, encoded_df2], axis=1)

In [ ]:
# Nu gaan we de Boolean kolommen omzetten naar 0 of 1

# omzetten van Scholarship, Hipertension, Diabetes, Alcoholism, Handcap en SMS_received naar 0 of 1
dataset['Scholarship'] = dataset['Scholarship'].astype(bool).astype(int)
dataset['Hipertension'] = dataset['Hipertension'].astype(bool).astype(int)
dataset['Diabetes'] = dataset['Diabetes'].astype(bool).astype(int)
dataset['Alcoholism'] = dataset['Alcoholism'].astype(bool).astype(int)
dataset['Handcap'] = dataset['Handcap'].astype(bool).astype(int)
dataset['SMS_received'] = dataset['SMS_received'].astype(bool).astype(int)
# TARGET: omzetten van Showed_up naar 0 of 1
dataset['Showed_up'] = dataset['Showed_up'].astype(bool).astype(int)

# Calculate new datediff column ## Niet nodig want de originele kolom is correct :-) 
# dataset['DateDiff2'] = (dataset['AppointmentDay'] - dataset['ScheduledDay']).dt.days


In [ ]:
# hier gaan de kolom 'Neighbourhood' ook encoderen ! 
# dit maakt van de ene kolom opeens 81 kolommen, dit is niet altijd de beste oplossing
# Neighbourhood omzetten met OHE via get dummies
dataset = pd.get_dummies(dataset, columns=['Neighbourhood'], drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dataset['Age'] = scaler.fit_transform(dataset[['Age']])

In [ ]:
# nieuwe X en y
# maar enkelen kolommen droppen die niet nodig zijn

X = dataset.drop(columns=['Gender', 'PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 'Showed_up'])
y = dataset['Showed_up']

In [ ]:
# Hier gaat het gebeuren, meerdere modellen vergelijken
# en dit jaar gaan we voor de volledige dataset met 100+ kolommen

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Functie om modellen te trainen en evalueren
def evaluate_models(X, y):
    # Schaal de features met StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # Om zeker te zijn dat alle data 'scaled' is

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Definieer modellen
    models = {
        "Logistic Regression": LogisticRegression(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "Extra Trees": ExtraTreesClassifier(),
        "AdaBoost": AdaBoostClassifier(),
        "Gradient Boosting": GradientBoostingClassifier(),
        "XGBoost": XGBClassifier(eval_metric='logloss'),
        "LGBM": LGBMClassifier(),
#        "SVM": SVC(probability=True),  # Dit duurde veel te lang om uit te voeren, deze is niet geschikt voor grotere datsets, daarom de volgende LinearSVC
#        "Linear SVM": LinearSVC(max_iter=10000), # zelfs 10000 iterations failed to converge, daarom de eruit gehaald
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "Neural Network": MLPClassifier(max_iter=500), # Deze duurt ookiets te lang om goed te zijn
    }

    # Resultaten opslaan
    scores = {}

    # Train en evalueer elk model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Bereken de score
        acc = accuracy_score(y_test, y_pred)
        scores[name] = acc
        print(f"{name}: {acc:.4f}")  # Print de score

    # Plot de scores
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=list(scores.keys()), y=list(scores.values()), palette="viridis")
        # Voeg de waarden boven de bars toe
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.4f}',  # Waarde boven de bar
                    (p.get_x() + p.get_width() / 2., p.get_height()),  # Positie van de annotatie
                    ha='center', va='center',  # Horizontaal en verticaal uitlijnen
                    fontsize=12, color='black',  # Stijl van de tekst
                    xytext=(0, 5),  # Plaatsing van de tekst boven de bar
                    textcoords='offset points')
    # Titels en labels toevoegen
    plt.xlabel("Model")
    plt.ylabel("Accuracy")
    plt.title("Vergelijking van Model Prestaties")
    plt.xticks(rotation=30, ha='right')  # Draai de x-as labels
    plt.ylim(0, 1) # Zet de limieten van de y-as
    plt.tight_layout()  # Zorg ervoor dat alles netjes past
    save_fig("Model_Prestaties_Full")
    plt.show()

# Start the engines !!
evaluate_models(X, y)


### SMOTE    
Het gebruik van SMOTE (Synthetic Minority Over-sampling Technique) zou hier zeker interessant kunnen zijn, omdat we wel te maken met een imbalance dataset, waar de targetkolom een sterke klasse-imbalance vertoont (bijvoorbeeld als 90% van de waarden 'False' zijn en maar 10% 'True'), kan SMOTE een oplossing zijn om deze onbalans te verhelpen.   
Hier spreken nvan 80/20, dus toch de moeite om eens uit te proberen.
  

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split de data in train en test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Schaal de features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Pas SMOTE toe om de balans te herstellen
smote = SMOTE(random_state=seed)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# Train je model met de resampled data
lr = LogisticRegression()
lr.fit(X_train_res, y_train_res)

# Evalueer je model
y_pred_smote_lr = lr.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_smote_lr)
print(f"Model accuracy after applying SMOTE: {accuracy:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train your model
rfc = RandomForestClassifier()
rfc.fit(X_train_res, y_train_res)

# Predict on test data
y_pred_smote_rfc = rfc.predict(X_test)

# Evaluate performance
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_smote_rfc)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred_smote_rfc))
print("ROC-AUC Score:", roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1]))

In [ ]:
import xgboost as xgb

# Maak en train het XGBoost-model
model_xgb = xgb.XGBClassifier(eval_metric='logloss')
model_xgb.fit(X_train_res, y_train_res)

# Voorspel de uitkomsten op de testset
y_pred = model_xgb.predict(X_test_scaled)

# Evalueer de nauwkeurigheid
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy after applying SMOTE: {accuracy:.4f}")


### GridSearchCV voor Random Forest    


In [ ]:
# voor de hieronder uit te voeren code, ga ik verder zonder de OHE van de kolom 'Neighbourhood'
features = ['Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Date.diff', 
            'ScheduledDayOfWeek_Monday', 'ScheduledDayOfWeek_Tuesday', 'ScheduledDayOfWeek_Wednesday', 'ScheduledDayOfWeek_Thursday', 
            'ScheduledDayOfWeek_Friday', 'ScheduledDayOfWeek_Saturday', 'AppointmentDayOfWeek_Monday', 'AppointmentDayOfWeek_Tuesday', 
            'AppointmentDayOfWeek_Wednesday', 'AppointmentDayOfWeek_Thursday', 'AppointmentDayOfWeek_Friday', 'AppointmentDayOfWeek_Saturday'
            ]
target = 'Showed_up'
X = dataset[features]
y = dataset[target]

# Split de data in train en test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definieer je model
model = RandomForestClassifier(random_state=seed)

# Definieer het grid van hyperparameters
param_grid = {
    'n_estimators': [50, 100, 150],    # Aantal bomen in het bos
    'max_depth': [5, 10],             # Diepte van de bomen  # Geen 'None' om te vermijden dat bomen te groot worden
    'min_samples_split': [2, 5, 10],   # Minimale aantal samples om een split te maken
    'min_samples_leaf': [1, 2, 4],     # Minimale aantal samples in een blad
    'max_features': ['sqrt', 'log2'],  # Maximale aantal features om te splitsen # 'auto' vermijden
}

# GridSearchCV instellen
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Voer GridSearch uit
grid_search.fit(X_train, y_train)

# Beste hyperparameters weergeven
print(f"Beste hyperparameters: {grid_search.best_params_}")

# Het beste model verkrijgen
best_model = grid_search.best_estimator_

# Model evalueren
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {acc:.4f}")


Nieuwe poging rfc

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definieer je model
rfc_model = RandomForestClassifier(class_weight='balanced', criterion='gini', random_state=seed)

# Definieer het grid van hyperparameters
param_grid_rfc = {
    'n_estimators': [50, 100, 150],    # Aantal bomen in het bos
    'max_depth': [2, 3, 6],             # Diepte van de bomen  # Geen 'None' om te vermijden dat bomen te groot worden
    'min_samples_split': [2, 10],      # Minimale aantal samples om een split te maken
    'min_samples_leaf': [1, 2],     # Minimale aantal samples in een blad
    'max_features': ['sqrt'],  # Maximale aantal features om te splitsen # 'auto' vermijden # 'log2' ook weg gelaten
}

# GridSearchCV instellen
gs_rfc = GridSearchCV(estimator=rfc_model, param_grid=param_grid_rfc, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Voer GridSearch uit
gs_rfc.fit(X_train, y_train)

# Beste hyperparameters weergeven
best_params_rfc = gs_rfc.best_params_

# Het beste model verkrijgen
best_model = gs_rfc.best_estimator_

# Model evalueren
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Beste hyperparameters: {best_params_rfc}")
print(f"Model accuracy: {acc:.4f}")


# Resultaten van alle combinaties weergeven
results_df_rfc = pd.DataFrame(gs_rfc.cv_results_)
results_df_rfc.to_csv(output_path / 'HNSA_1_grid_search_RFC_results.csv', index=False)

# Laat een overzicht zien van de top 5 van de resultaten
print("\nTop 5 combinaties van hyperparameters:")
print(results_df_rfc[['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 
                      'param_min_samples_leaf', 'param_max_features', 'mean_test_score', 'std_test_score']].sort_values(by='mean_test_score', ascending=False).head())

# Visualiseer de prestaties van de verschillende combinaties als een heatmap
pivot_table_rfc = results_df_rfc.pivot_table(values='mean_test_score', 
                                     index='param_max_depth', 
                                     columns='param_n_estimators')

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table_rfc, annot=True, cmap='viridis', fmt='.4f', linewidths=0.5)
plt.title("Grid Search Resultaten: Mean Test Score per Max Depth & N Estimators")
plt.xlabel('Aantal Estimators')
plt.ylabel('Max Diepte')
save_fig("HNSA_1_Grid_Search_Heatmap_RandomForrest")
plt.show()

nieuwe poging xgb

In [ ]:
from sklearn.model_selection import GridSearchCV

# Dit is een classificatie problem en we kiezen hier voor de XG Boost Classifier
from xgboost import XGBClassifier

# Waarschuwingen onderdrukken
warnings.simplefilter("ignore", category=UserWarning)

# Definitie van het model
xgb_model = XGBClassifier(eval_metric="logloss", random_state = seed)

# Definieer het grid van hyperparameters
param_grid_xgb = {
    'n_estimators': [100, 200, 500],     # Aantal bomen in het bos
    'max_depth': [3, 6, 9],              # Diepte van de bomen  # Geen 'None' om te vermijden dat bomen te groot worden
    "learning_rate" : [0.01, 0.1, 0.2]   # Snelheid van leren. Verlaagt de bijdrage van elke boom., meestal tss 0.01 en 0.2, kleine waarde vereisen meer bomen
}

# make a GridSearchCV object
gs_xgb = GridSearchCV(estimator = xgb_model,
                  param_grid = param_grid_xgb,
                  scoring = 'accuracy', #sklearn.metrics.SCORERS.keys() to score
                  cv = 3, # cross validation
                  n_jobs = 1,
                  verbose = 9 # hoeveel info wil je zien
                  )
# Voer GridSearch uit
gs_xgb.fit(X_train, y_train)


# Beste parameters en score
best_params = gs_xgb.best_params_
best_score = gs_xgb.best_score_
print(f"Beste parameters: {best_params}")
print(f"Beste model accuracy: {best_score:.4f}")

# Resultaten verzamelen in een DataFrame
results_df_xgb = pd.DataFrame(gs_xgb.cv_results_)
results_df_xgb.to_csv(output_path / 'grid_search_XGB_results.csv', index=False)

# Maak een pivot table voor de heatmap
pivot_table_xgb = results_df_xgb.pivot_table(values='mean_test_score', 
                                     index='param_max_depth', 
                                     columns='param_n_estimators')

# Heatmap plotten
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table_xgb, annot=True, cmap='viridis', fmt='.4f', linewidths=0.5)
plt.title("Grid Search Resultaten: Mean Test Score voor XGBoost")
plt.xlabel('Aantal Estimators')
plt.ylabel('Max Diepte')
save_fig("HNSA_1_Grid_Search_Heatmap_XGBoost")
plt.show()

In [ ]:
# Bereken de uitvoertijd, vergeet niet om de start_time te definieren in het begin van het  script
end_time = datetime.now()
#start_time = datetime.datetime.now()
print(f"Gestart om: {start_time}\n")
print(f"Eindtijd: {end_time}\n")
# print uitvoertijd in seconden
print(f"Uitvoertijd: {end_time-start_time}\n")
# print THE END in grote karakters

import pyfiglet
text = "THE  END"
ascii_art = pyfiglet.figlet_format(text)
print(ascii_art)